In [1]:
import pandas as pd
import sys
sys.path.insert(1, '../scripts')
from preprocessor import remove_emojies
from pridicter import get_price
from tqdm import tqdm
tqdm.pandas()


/home/est-lul/Documents/Projects/telegram-commerce-ner/.venv/lib64/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
df = pd.read_csv('../../data/telegram_data.csv')
df = df.dropna(subset=['Message'])
df = df.drop_duplicates(subset=['Message'])

In [3]:
df['Message'] = df['Message'].progress_apply(remove_emojies)

100%|██████████| 14210/14210 [00:00<00:00, 101896.61it/s]


In [4]:
df.sample(10)

,Channel Title,Channel Username,ID,Message,Date,Media Path,Views
7727,Sheger online-store,@Shageronlinestore,2753,Double Layer Rotating Candy Box\n\nStore seeds...,2023-11-30 05:01:16+00:00,NaN,13736.0
30979,ምርጥ ዕቃ,@MerttEka,5240,Toothbrush Sterilizer\n\n ይሄንን t.me/MerttEka ...,2024-03-19 21:38:04+00:00,NaN,31984.0
5479,Sheger online-store,@Shageronlinestore,5160,ለክቡራን ደንበኞቻችን \n\nእናንተን በቅርበት ለማገልገል \n\n#በለቡ ...,2024-08-30 14:49:31+00:00,NaN,20562.0
15342,SINA KIDS/ሲና ኪድስⓇ,@sinayelj,5633,Brand BERR\n25-30 ቀጥር አላቸው\n1350 ብር\n090570744...,2022-04-12 20:01:37+00:00,NaN,16798.0
20435,ማራኪ ცЯﾑŋの™,@marakibrand,7086,ADIDAS ™ \n Made In VIETNAM \n\n Size: #41 #4...,2022-04-11 12:59:54+00:00,NaN,15273.0
21719,ማራኪ ცЯﾑŋの™,@marakibrand,5297,NIKE ™ tracksuit\n Made In TURKEY \n\n Size: M...,2020-10-16 03:07:37+00:00,NaN,15707.0
33625,ምርጥ ዕቃ,@MerttEka,2134,Tea and Coffee Set\n የቡናና ሻይ ማፍሊያ እና ማቅረቢያ\n የ...,2022-06-21 08:39:37+00:00,NaN,31732.0
32159,ምርጥ ዕቃ,@MerttEka,3826,U Shaped Baby Toothbrush\n\n የልጆች የጥርስ ቡሩሽ\n አ...,2023-08-14 16:15:57+00:00,NaN,32804.0
32866,ምርጥ ዕቃ,@MerttEka,3042,FAUCET QUICK CUP WASHER\n የብርጭቆ ማጠቢያ\n ቧንቧ መስመ...,2022-12-30 16:47:27+00:00,NaN,25394.0
867,ልዩ እቃ,@Leyueqa,6653,36 pairs over the door shoe rack\n\n በቀላሉ የሚገጣ...,2025-02-04 06:06:56+00:00,NaN,23132.0


In [5]:
df['Date'] = pd.to_datetime(df['Date'])
df['Week'] = df['Date'].dt.isocalendar().week
df['Year'] = df['Date'].dt.isocalendar().year

In [6]:
# Count messages per vendor per week
weekly_counts = df.groupby(['Channel Username', 'Year', 'Week'])['Message'].count().reset_index()
weekly_counts = weekly_counts.rename(columns={'Message': 'posts_per_week'})

# Calculate average posts per week for each vendor
avg_posts_per_week = weekly_counts.groupby('Channel Username')['posts_per_week'].mean().reset_index()
avg_posts_per_week = avg_posts_per_week.rename(columns={'posts_per_week': 'avg_posts_per_week'})

In [7]:
avg_posts_per_week

,Channel Username,avg_posts_per_week
0,@Leyueqa,10.556452
1,@MerttEka,15.500000
2,@Shageronlinestore,13.801075
3,@marakibrand,8.448925
4,@qnashcom,6.575893
5,@sinayelj,6.602941


In [8]:
# Group by vendor
vendor_stats = df.groupby(['Channel Username']).agg({
    'Views': 'mean'
}).rename(columns={
    'Views': 'average_views'})

vendor_stats['average_posts_per_week'] = avg_posts_per_week.set_index('Channel Username')['avg_posts_per_week']

In [9]:
vendor_stats

,average_views,average_posts_per_week
Channel Username,,
@Leyueqa,37415.465241,10.556452
@MerttEka,25323.737360,15.500000
@Shageronlinestore,11811.526685,13.801075
@marakibrand,18876.353802,8.448925
@qnashcom,25524.465037,6.575893
@sinayelj,15358.201930,6.602941


In [ ]:
df['price'] = df['Message'].progress_apply(get_price)

In [ ]:
avg_price = df.groupby('Channel Username')['price'].mean().rename('avg_price').reset_index()
vendor_stats = vendor_stats.merge(avg_price, on='Channel Username', how='left')

In [ ]:
vendor_stats["lending_score"] = (
    vendor_stats['avg_views'] * 0.5 +
    vendor_stats['avg_posts_per_week'] * 0.3 +
    vendor_stats['avg_price'] * 0.2
)